## Deliverable 2. Create a Customer Travel Destinations Map.

In [98]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [99]:
# 1. Import the WeatherPy_database.csv file (?). 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Riacho De Santana,BR,-13.6092,-42.9389,72.36,88,100,3.69,moderate rain
1,1,Bol,TD,13.4586,14.7147,72.82,34,0,15.66,clear sky
2,2,Saint-Philippe,RE,-21.3585,55.7679,78.39,80,99,15.50,overcast clouds
3,3,Dingle,PH,10.9995,122.6711,73.38,89,63,8.48,broken clouds
4,4,Kaniama,CD,-7.5667,24.1833,66.49,96,100,4.05,light rain


In [100]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [114]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp)&
                                       (city_data_df["Max Temp"]<= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Riacho De Santana,BR,-13.6092,-42.9389,72.36,88,100,3.69,moderate rain
1,1,Bol,TD,13.4586,14.7147,72.82,34,0,15.66,clear sky
2,2,Saint-Philippe,RE,-21.3585,55.7679,78.39,80,99,15.50,overcast clouds
3,3,Dingle,PH,10.9995,122.6711,73.38,89,63,8.48,broken clouds
6,6,Ancud,CL,-41.8697,-73.8203,75.22,66,10,6.38,clear sky
...,...,...,...,...,...,...,...,...,...,...
687,687,Orange Cove,US,36.6244,-119.3137,71.62,34,0,3.98,clear sky
688,688,Envira,BR,-7.3000,-70.2167,77.92,94,97,2.84,light rain
689,689,Monte Carmelo,BR,-18.7247,-47.4986,70.32,91,100,6.42,overcast clouds
695,695,Keffi,NG,8.8486,7.8736,82.31,15,31,2.95,scattered clouds


In [115]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [116]:
null_cities = preferred_cities_df[pd.isnull(preferred_cities_df).any(axis=1)]
null_cities
#Upon google searches. all four cities are from Namibia.
#The two-letter country abbreviation for Namibia is NA, the three-letter code is NAM
#They are not empty rows. 

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
132,132,Mariental,NaN,-24.6333,17.9667,83.89,34,100,4.32,overcast clouds
326,326,Gobabis,NaN,-22.4500,18.9667,71.87,57,48,6.71,scattered clouds
440,440,Walvis Bay,NaN,-22.9575,14.5053,72.55,78,100,6.91,overcast clouds
570,570,Opuwo,NaN,-18.0607,13.8400,74.61,41,100,6.38,overcast clouds


In [119]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# As they are not empty rows, I decided to replace country code NA with NAM for the four cities
replace_cities_df=preferred_cities_df.fillna("NAM")
nam_cities = replace_cities_df.loc[replace_cities_df["Country"]== "NAM"]
nam_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
132,132,Mariental,NAM,-24.6333,17.9667,83.89,34,100,4.32,overcast clouds
326,326,Gobabis,NAM,-22.4500,18.9667,71.87,57,48,6.71,scattered clouds
440,440,Walvis Bay,NAM,-22.9575,14.5053,72.55,78,100,6.91,overcast clouds
570,570,Opuwo,NAM,-18.0607,13.8400,74.61,41,100,6.38,overcast clouds


In [120]:
# double check to make sure there is no empty rows
replace_cities_df.count()

City_ID                260
City                   260
Country                260
Lat                    260
Lng                    260
Max Temp               260
Humidity               260
Cloudiness             260
Wind Speed             260
Current Description    260
dtype: int64

In [122]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = replace_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Riacho De Santana,BR,72.36,moderate rain,-13.6092,-42.9389,
1,Bol,TD,72.82,clear sky,13.4586,14.7147,
2,Saint-Philippe,RE,78.39,overcast clouds,-21.3585,55.7679,
3,Dingle,PH,73.38,broken clouds,10.9995,122.6711,
6,Ancud,CL,75.22,clear sky,-41.8697,-73.8203,
9,Kavieng,PG,81.16,broken clouds,-2.5744,150.7967,
10,Grand Gaube,MU,79.38,overcast clouds,-20.0064,57.6608,
11,Touros,BR,86.90,clear sky,-5.1989,-35.4608,
12,Xambioa,BR,83.59,overcast clouds,-6.4111,-48.5364,
13,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,


In [123]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found...skipping.')
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [124]:
hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Riacho De Santana,BR,72.36,moderate rain,-13.6092,-42.9389,Casa Nogueira
1,Bol,TD,72.82,clear sky,13.4586,14.7147,Hotel 3 Etoiles
2,Saint-Philippe,RE,78.39,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
3,Dingle,PH,73.38,broken clouds,10.9995,122.6711,SEGAYA PROPERTY
6,Ancud,CL,75.22,clear sky,-41.8697,-73.8203,Hotel Arena Gruesa
...,...,...,...,...,...,...,...
687,Orange Cove,US,71.62,clear sky,36.6244,-119.3137,
688,Envira,BR,77.92,light rain,-7.3000,-70.2167,
689,Monte Carmelo,BR,70.32,overcast clouds,-18.7247,-47.4986,Park hotel
695,Keffi,NG,82.31,scattered clouds,8.8486,7.8736,Imam Family


In [129]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
na_hotel_df = hotel_df.replace("", np.nan)
clean_hotel_df = na_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Riacho De Santana,BR,72.36,moderate rain,-13.6092,-42.9389,Casa Nogueira
1,Bol,TD,72.82,clear sky,13.4586,14.7147,Hotel 3 Etoiles
2,Saint-Philippe,RE,78.39,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
3,Dingle,PH,73.38,broken clouds,10.9995,122.6711,SEGAYA PROPERTY
6,Ancud,CL,75.22,clear sky,-41.8697,-73.8203,Hotel Arena Gruesa
...,...,...,...,...,...,...,...
679,Sur,OM,70.79,clear sky,22.5667,59.5289,Sur Plaza Hotel
683,Rio Gallegos,AR,78.85,broken clouds,-51.6226,-69.2181,Hotel Santa Cruz
689,Monte Carmelo,BR,70.32,overcast clouds,-18.7247,-47.4986,Park hotel
695,Keffi,NG,82.31,scattered clouds,8.8486,7.8736,Imam Family


In [130]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [131]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [132]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))